# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import math
import xlsxwriter

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [3]:
drive = 'D'
raw_fold = f'{drive}:/data/big2/BD1/raw'
ods_fold = f'{drive}:/data/big2/ODS'
sample_fold = f'{drive}:/data/big2/BD1/sample'
analysis_fold = f'{drive}:/data/big2/BD1/an'

# Load

## 병합 result 최종 파일

In [4]:
# about 18.9s
name = '4등급_차량_리스트_양식_result(전체)_Grade_추가(2023.04.18_PM.03.07.04)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 37 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   차대번호        1502235 non-null  object 
 1   차량번호        1502235 non-null  object 
 2   제원관리번호      1502235 non-null  object 
 3   차종          1502235 non-null  object 
 4   용도          1502235 non-null  object 
 5   최초등록일       1502235 non-null  int64  
 6   연식          1502235 non-null  int64  
 7   제작일자        1335178 non-null  float64
 8   검사유효일       1502223 non-null  float64
 9   배출가스인증번호    1384783 non-null  object 
 10  등급          1502235 non-null  int64  
 11  차명          1502235 non-null  object 
 12  차종분류        1502235 non-null  object 
 13  차종유형        1502235 non-null  object 
 14  자동차형식       1501958 non-null  object 
 15  제작사명        1492590 non-null  object 
 16  연료          1502235 non-null  object 
 17  엔진형식        1500400 non-null  object 
 18  총중량         1502234 no

# 전처리

In [5]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade'],
      dtype='object')

In [6]:
df['연료'].value_counts(dropna=False)

경유             1153813
휘발유             323019
LPG(액화석유가스)      25400
알코올                  1
기타연료                 1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [7]:
# 한불모터스에서 확인 한 1대 3등급으로 변경
# 3등급 : 4396 + 1 = 4397
df['등급_수정'].value_counts(dropna=False)

4.00    1497711
3.00       4397
5.00        127
Name: 등급_수정, dtype: int64

In [8]:
df.loc[df['제작사명'] == '현대자동차(주)', '배인번호_수정'].unique()

array(['5MY-HD-24-35', '6MY-HD-14-62', '5MY-HD-24-56', '5MY-HD-24-59',
       '6MY-HD-14-63', '6MY-HD-14-64', '5MY-HD-24-55', '6MY-HD-14-96',
       '6MY-HD-14-97', '7MY-HD-14-57', '6MY-HD-24-100', '6MY-HD-24-12',
       '7MY-HD-14-45', '6MY-HD-24-44', '7MY-HD-13-58', '7MY-HD-14-46',
       '7MY-HD-14-47', '7MY-HD-14-68', '7MY-HD-23-01', '7MY-HD-24-08',
       '7MY-HD-24-09', '7MY-HD-24-28', '7MY-HD-24-29', '7MY-HD-24-30',
       '7MY-HD-24-31', '9MY-HD-14-21', '확인불가', nan, 'CMY-HD-14-11',
       '6MY-HD-23-55', '3MY-HD-14', '3MY-HD-15'], dtype=object)

In [9]:
len(df.loc[df['제작사명'] == '현대자동차(주)', '배인번호_수정'].unique())

32

In [10]:
df.loc[df['제작사명'] == '기아자동차(주)', '배인번호_수정'].unique()

array(['5MY-KM-24-17', '5MY-KM-24-57', '5MY-KM-24-64', '6MY-KM-14-57',
       '6MY-KM-14-58', '6MY-KM-14-59', '6MY-KM-14-61', '6MY-KM-14-60',
       '6MY-KM-24-06', '6MY-KM-24-81', '7MY-KM-14-52', '4MY-KM-08',
       '7MY-KM-14-63', '7MY-KM-14-64', '7MY-KM-14-65', '7MY-KM-24-32',
       '8MY-KM-13-19', '8MY-KM-14-21', '8MY-KM-14-29', '9MY-KM-14-22',
       '9MY-KM-14-41', '7MY-KM-24-33', '확인불가', nan, '4MY-KM-06',
       '4MY-KM-07', '5MY-KM-24-32'], dtype=object)

In [12]:
len(df.loc[df['제작사명'] == '기아자동차(주)', '배인번호_수정'].unique())

27

# 코드 마지막